In [21]:
cd /Users/wj/crism_ml

/Users/wj/crism_ml


In [22]:
import os
import scipy.io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from crism_ml.io import load_image
from crism_ml.io import image_shape
from crism_ml.train import compute_bland_scores
from crism_ml.preprocessing import filter_bad_pixels
from crism_ml.preprocessing import remove_spikes_column
from crism_ml.train import feat_masks, train_model_bland
from sklearn.metrics import confusion_matrix, classification_report

In [23]:
test_df = pd.read_pickle('test_df.pkl')
test_df

,image_id,x,y,0,1,2,3,4,5,6,...,429,430,431,432,433,434,435,436,437,label
0,129D1,42,234,0.183240,0.183240,0.177040,0.181723,0.176845,0.182095,0.176937,...,0.390600,0.394394,0.398004,0.402821,0.183240,0.183240,0.183240,0.183240,0.0,1
1,129D1,33,156,0.183240,0.183240,0.179713,0.177953,0.174137,0.177956,0.171644,...,0.415097,0.417611,0.421339,0.430918,0.183240,0.183240,0.183240,0.183240,0.0,1
2,129D1,78,78,0.183240,0.183240,0.177950,0.170932,0.168615,0.170199,0.166566,...,0.401058,0.406532,0.407522,0.410457,0.419562,0.183240,0.183240,0.183240,0.0,1
3,129D1,42,147,0.183240,0.183240,0.179855,0.182989,0.178902,0.182606,0.177878,...,0.423789,0.430713,0.436174,0.441805,0.183240,0.183240,0.183240,0.183240,0.0,1
4,129D1,42,235,0.183240,0.183240,0.172990,0.176540,0.171127,0.176303,0.171870,...,0.377796,0.379615,0.383085,0.388312,0.183240,0.183240,0.183240,0.183240,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542781,0285A,261,479,0.168257,0.168257,0.215043,0.223787,0.219751,0.219945,0.220625,...,0.250695,0.253981,0.258225,0.259552,0.256002,0.168257,0.168257,0.168257,0.0,0
542782,0285A,262,479,0.168257,0.168257,0.211554,0.218031,0.214185,0.214958,0.215098,...,0.266204,0.269851,0.274947,0.277691,0.274571,0.168257,0.168257,0.168257,0.0,0
542783,0285A,263,479,0.168257,0.168257,0.202948,0.208156,0.205130,0.206592,0.205919,...,0.252587,0.253365,0.257215,0.261810,0.262511,0.168257,0.168257,0.168257,0.0,0
542784,0285A,264,479,0.168257,0.168257,0.208354,0.214310,0.211355,0.213511,0.212933,...,0.252287,0.253646,0.256610,0.261630,0.260693,0.168257,0.168257,0.168257,0.0,0


In [24]:
len(test_df[test_df['label'] == 1])

65584

In [25]:
test_images = test_df['image_id'].unique()
print(len(test_images))
print(test_images)

68
['129D1' '0D4EC' '1B804' '0AC95' '20B9D' '13C94' '0C008' '1ECAD' '0CAB2'
 '0C23D' '0BEC0' '0A4FC' '233AC' '0A2C2' '0B385' '096BF' '23FB0' '1B8C9'
 '0BAB3' '39990' '0AD3D' '19AA0' '1654F' '247A0' '0A546' '03FB9' '20364'
 '17D51' '16C19' '0B6A2' '13EE8' '040FF' '22003' '1187B' '1E26E' '12E48'
 '136D3' '0A425' '2038F' '0BDD9' '0D364' '1EBC6' '1DABB' '121D0' '0CEF7'
 '0CE1D' '0863E' '1FD35' '11D66' '07BC8' '24C1A' '21D02' '11106' '1217E'
 '1CA91' '13039' '13E4D' '17DBF' '0ACE6' '0BABA' '0A053' '12E72' '10934'
 '23543' '176BD' '0D349' '02885' '0285A']


In [26]:
data = scipy.io.loadmat('datasets/CRISM_bland_unratioed.mat')

def get_id(x):
    return x[0][:5]
vfunc = np.vectorize(get_id)
im_names = np.char.upper(vfunc(data['im_names']).flatten().astype(str))
pixims = data['pixims'].flatten()                  
coordinates = data['pixcrds']
spectra = data['pixspec']

bland_df_data = pd.DataFrame({
    'numerical_id': pixims,
    'x': coordinates[:, 0],
    'y': coordinates[:, 1],
})

spectra_df = pd.DataFrame(spectra, columns=[f'{i+1}' for i in range(spectra.shape[1])])
bland_df_data = pd.concat([bland_df_data, spectra_df], axis=1)

image_id_mapping = dict(enumerate(im_names, start=1))
bland_df_data['image_id'] = bland_df_data['numerical_id'].map(image_id_mapping)
bland_df_data

,numerical_id,x,y,1,2,3,4,5,6,7,...,342,343,344,345,346,347,348,349,350,image_id
0,1,89,215,0.126150,0.123324,0.123346,0.122697,0.122847,0.122415,0.121558,...,0.098996,0.101664,0.103983,0.105582,0.107155,0.109767,0.112016,0.114639,0.117502,0289E
1,1,74,423,0.149629,0.147174,0.147389,0.146957,0.146764,0.146358,0.146254,...,0.103207,0.105128,0.107558,0.109991,0.112132,0.113990,0.116037,0.118174,0.120477,0289E
2,1,54,155,0.125054,0.123151,0.122999,0.122891,0.123414,0.122534,0.122217,...,0.087160,0.089292,0.091167,0.093757,0.096027,0.097236,0.098396,0.099665,0.101979,0289E
3,1,89,216,0.127114,0.124607,0.124518,0.124081,0.124266,0.123949,0.123404,...,0.097394,0.099989,0.102057,0.103600,0.105445,0.107787,0.109771,0.111487,0.113518,0289E
4,1,50,358,0.129833,0.128444,0.127840,0.127635,0.127289,0.127189,0.126209,...,0.110896,0.112905,0.115901,0.118805,0.121005,0.122814,0.123849,0.124782,0.127021,0289E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337612,340,95,27,0.160413,0.153373,0.157163,0.158137,0.157017,0.155377,0.158092,...,0.077179,0.081959,0.085725,0.087799,0.087353,0.086721,0.087042,0.087762,0.090625,39990
337613,340,56,34,0.154073,0.150957,0.151212,0.153644,0.152609,0.152407,0.153543,...,0.082212,0.084375,0.085852,0.087713,0.091485,0.093344,0.093971,0.094584,0.094276,39990
337614,340,48,32,0.162860,0.158374,0.159284,0.161899,0.158506,0.159438,0.159314,...,0.062471,0.065045,0.065833,0.062852,0.061609,0.061640,0.062956,0.064604,0.068007,39990
337615,340,33,26,0.257411,0.256771,0.258491,0.265441,0.256465,0.264001,0.264152,...,0.116020,0.113713,0.115261,0.119238,0.124086,0.128880,0.128196,0.126115,0.122044,39990


In [27]:
selected_rows = bland_df_data.loc[bland_df_data['image_id'].isin(test_images)]
selected_indices = selected_rows.index
print(len(selected_indices))
print(selected_indices)

65584
Index([  5000,   5001,   5002,   5003,   5004,   5005,   5006,   5007,   5008,
         5009,
       ...
       337607, 337608, 337609, 337610, 337611, 337612, 337613, 337614, 337615,
       337616],
      dtype='int64', length=65584)


In [28]:
pixims_filtered = np.delete(pixims, selected_indices)
spectra_filtered = np.delete(spectra, selected_indices, axis=0)

print(len(pixims_filtered))
print(len(spectra_filtered))

272033
272033


In [29]:
filtered_data = {
    'pixims': pixims_filtered,
    'pixspec': spectra_filtered
}

scipy.io.savemat('my_data/CRISM_bland_unratioed.mat', filtered_data)

In [30]:
fin0, fin = feat_masks()
model_retrain = train_model_bland('my_data', fin0)

In [31]:
def create_image_paths(directory_path):
    image_paths = {}
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.lower().endswith('.hdr'):
                id = file[6:11].upper()
                image_paths[id] = os.path.join(root, file)
    return image_paths

In [32]:
image_paths = create_image_paths('/Users/wj/Desktop/CRISM_DATA')

id = []
path = []
for im_id, im_path in image_paths.items():
    if im_id in test_images:
        id.append(im_id)
        path.append(im_path)
print(len(id))
print(len(path))

68
68


In [33]:
result_retrain = []
result_origin = []

grouped = test_df.groupby('image_id')

for i in range(len(test_images)):

    im_id = id[i]
    im_path = path[i]

    mat = load_image(im_path)
    if_, rem = filter_bad_pixels(mat['IF'])

    im_shape = image_shape(mat)
    
    if1 = remove_spikes_column(if_.reshape(*im_shape, -1), 3, 5).reshape(if_.shape)
    slog_retrain = compute_bland_scores(if1, (model_retrain, fin0))

    group = grouped.get_group(im_id)

    for _, row in group.iterrows():
        x, y = row['x'], row['y']
        index = np.where((mat['x'] == x) & (mat['y'] == y))[0]
        score_retrain = slog_retrain[index][0]
        result_retrain.append([im_id, x, y, score_retrain])


result_df_retrain = pd.DataFrame(result_retrain, columns=['image_id', 'x', 'y', 'score'])

In [34]:
print(result_df_retrain)

       image_id    x    y         score
0         129D1   42  234 -23120.606674
1         129D1   33  156 -22336.940677
2         129D1   78   78 -22274.364024
3         129D1   42  147 -23050.046034
4         129D1   42  235 -23870.383029
...         ...  ...  ...           ...
169255    0285A  261  479 -17633.189845
169256    0285A  262  479 -17057.586390
169257    0285A  263  479 -16978.022007
169258    0285A  264  479 -17226.919190
169259    0285A  265  479 -17173.200304

[169260 rows x 4 columns]


In [35]:
print(len(result_df_retrain['image_id'].unique()))

68


In [36]:
result_df_retrain = result_df_retrain.sort_values(by='score', ascending=False)

result_df_retrain['label'] = 0

label_column_index = result_df_retrain.columns.get_loc('label')
result_df_retrain.iloc[:len(test_df[test_df['label'] == 1]), label_column_index] = 1

result_df_reordered_retrain = result_df_retrain.set_index(['image_id', 'x', 'y'])
result_df_reordered_retrain = result_df_reordered_retrain.reindex(test_df.set_index(['image_id', 'x', 'y']).index)
result_df_reordered_retrain = result_df_reordered_retrain.reset_index()

result_df_reordered_retrain

,image_id,x,y,score,label
0,129D1,42,234,-23120.606674,0
1,129D1,33,156,-22336.940677,0
2,129D1,78,78,-22274.364024,0
3,129D1,42,147,-23050.046034,0
4,129D1,42,235,-23870.383029,0
...,...,...,...,...,...
169255,0285A,261,479,-17633.189845,0
169256,0285A,262,479,-17057.586390,0
169257,0285A,263,479,-16978.022007,0
169258,0285A,264,479,-17226.919190,0


In [37]:
import collections

collections.Counter(result_df_reordered_retrain['label'].values)

Counter({0: 103676, 1: 65584})

In [38]:
y_test = test_df['label'].values
y_pred = result_df_reordered_retrain['label'].values

confusion_mat = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Confusion Matrix:\n", confusion_mat)
print("\nClassification Report:\n", class_report)

Confusion Matrix:
 [[52705 50971]
 [50971 14613]]

Classification Report:
               precision    recall  f1-score   support

           0       0.51      0.51      0.51    103676
           1       0.22      0.22      0.22     65584

    accuracy                           0.40    169260
   macro avg       0.37      0.37      0.37    169260
weighted avg       0.40      0.40      0.40    169260

